# Simplified Bepro Model
## Introduction to Bepro

>Bepro Network is a decentralized marketplace and system that connects developers with operators and anyone looking to build open-source development repositories. The network is self-managed protocol, managed by the BEPRO token, a utility ERC20 token that enables token holders to manage disputes in the Network and earn token rewards by providing value.
https://docs.bepro.network/

There are basically eight mechanisms in which an agent can interact with an instance of BePro Network:

1. `createBounty`: open issues on the platform in exchange for ERC20 Tokens to developers;
2. `cancelBounty`: cancel the issue that was first created. If the bounty is canceled after the draft time period, the operation comes with a penalty of X% of the bounty;
3. `createPullRequest`: developers notify the operators they are solving the issue published;  
4. `cancelPullRequest`: developers notify the operators they are no longer solving the issue;
5. `markPullRequestAsReady`: developers notify the operators they solved the problem and are for the merge proposals;
6. `createBountyProposal`: any council, holder of 25M $BEPRO, can make a merge proposal for a resolution mechanism for a bounty. In case the same proposal is passed through the curation proccess, the council member receives X% of the bounty;
7. `disputeProposal`: council member contests a merge proposal with a new one;
8. `refuseBountyProposal`: bounty creator can refuse the bounty proposals (what is his power regarding this?);
9. `closeBounty`: proccess that takes place after the rewards are distributed to every agent accountable (including the treasury) and the bounty NFT is minted;


Fees
1. `Network Fee`: (5% currently): of the total bounty prize that goes to a treasury address for each bounty created on the system;
2. `Curator Fee` (3% currently): of the total bounty prize that goes to the Curator member that proposes the Mergee Distribution/Request;
3. `Cancel Fee` (1% currently): of the total bounty prize that goes to a treasury address when a bounty is canceled;
4. Study the possibility of new parameters and dynamic changes to the existing ones;

## Limitations and simplifications of this model
* **Bounties are either created or, canceled or closed**

* **Usage of the white-label by an operator doesn't require staking $BEPRO**

* **All bounties will be payed in $BEPRO** instead of any ERC20 tokens

* **Gas fees are counted as 0 or irrelevant for the network agents**;

* **User _behavior_ has not been modeled**. User actions are derived from the history of events of the Bepro instance being analyzed;

* **One timestep in the simulation will be equal to 24 hours**

* **Pull requests and merge proccess are abstracted**, i.e., every time a bounty is closed, the model assumes the proccess of dispute and curation has been made successfuly

---

## MODEL

### Initial conditions
These are the initial conditions of the [BEPRO Development DAO](https://development.bepro.network/)

In [213]:
# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

# Included with cadCAD
import pandas as pd

# For analytics
import numpy as np
# For visualization
import plotly.express as px

import math
import random

In [214]:
initial_state={
    'bounties_in_progress':52,
    'bounties_closed': 6,
    
    'bepro_in_network': 4884451,
    'treasury': 1393131475,
    
    'number_of_agents':10000
}

In [215]:
system_params = {
   
   'network_fee': [0.05],
   'curator_fee':[0.03],
   'cancel_fee':[0.01],
   
   'prob_create':[0.1],
   'prob_cancel':[0.01],
   'prob_close':[0.03]
}

print(round(10000*random.uniform(0.0001,0.0002),0))

2.0


In [216]:
# POLICY FUNCTIONS
def p_createBounty(params, substep, state_history, previous_state):
    aux = random.random()
    new_bounties=0
    bepro_ammount_added=0
    agents=previous_state['number_of_agents']
    agents_variation=0
    
    if aux<params['prob_create']:
        new_bounties += round(agents*random.uniform(0.0001,0.0002),0)  # Assumes the number of created bounties is proportional to the number of agents currently in the network
        bepro_ammount_added+=round(random.uniform(1000,1000000),-3) # Considers the total ammunt of new bounty rewards per day varies between 1000 and 100.000 $BEPRO, will add a Gama distribution derived from gitcoin bounties data
        agents_variation+=round(new_bounties,0) # Assumes 1 agent enters the network per bounty created
        
    return {
        'delta_create': new_bounties, 
        'delta_add_network':bepro_ammount_added,
        'agents_create':agents_variation
        }


def p_cancelBounty(params, substep, state_history, previous_state):
    aux = random.random()
    canceled_bounties=0
    cancelation_fees=0
    bepro_ammount_canceled=0
    agents=previous_state['number_of_agents']
    agents_variation=0
    
    if aux<params['prob_cancel']:
        canceled_bounties-=round(agents*random.uniform(0.0001,0.0005),0)  # Assumes the number of canceled bounties is proportional to the number of agents currently in the network
        bepro_ammount_canceled-=round(random.uniform(1000,1000000),-3) # Assumes the ammount of tokens the leave the network due to cancelation is a random number between 1k and 100k $BEPRO
        agents_variation-=round(canceled_bounties,0) # Assumes one agent cancels the network per bounty closed
        if random.random()<(3/60): # 3 days when cancelation is free vs 60 days while bounty will be up, obviously the probability is not this direct, it's more likely the cancelation will be done within the first 3 days then the rest of the 57
            cancelation_fees+=abs(bepro_ammount_canceled)*params['cancel_fee']
            
    return {
        'delta_cancel':canceled_bounties, 
        'delta_cancel_network':bepro_ammount_canceled ,
        'delta_cancel_network_fee':cancelation_fees,
        'agents_cancel':agents_variation
        }

def p_closeBounty(params, substep, state_history, previous_state):
    aux = random.random()
    bepro_ammount_merged=0
    closed_bounties=0
    network_fee=0
    agents=previous_state['number_of_agents']
    agents_variation=0
    
    if aux<params['prob_close']:
        closed_bounties-=round(agents*random.uniform(0.0001,0.0002),0) # Assumes the number of closed bounties is proportional to the number of agents currently in the network
        bepro_ammount_merged-=round(random.uniform(1000,1000000),-3)
        network_fee += round(params['network_fee']*abs( bepro_ammount_merged),0)
        agents_variation -= round(abs(closed_bounties),0) # Assumes one agent leaves the network per bounty closed
        
    return {
        'delta_close':closed_bounties, 
        'delta_close_network':bepro_ammount_merged,  
        'delta_add_network_fee':network_fee,
        'agents_close': agents_variation
        }
       
        
# STATE UPDATE FUNCTIONS
def s_bounties_progress(params, substep, state_history, previous_state,policy_input):
    cb=previous_state['bounties_in_progress']
    n=cb+policy_input['delta_create']+policy_input['delta_cancel']+policy_input['delta_close']
    return 'bounties_in_progress', max(0, n)

def s_bounties_closed(params, substep, state_history, previous_state,policy_input):
    bc=previous_state['bounties_closed']
    n=bc-policy_input['delta_close']
    return 'bounties_closed',n

def s_bepro_in_network(params, substep, state_history, previous_state,policy_input):
    network=previous_state['bepro_in_network']
    n=network+policy_input['delta_close_network']+policy_input['delta_add_network']+policy_input['delta_cancel_network']
    return 'bepro_in_network', max(0, n)

def s_bepro_in_treasury(params, substep, state_history, previous_state,policy_input):
    network=previous_state['treasury']
    n=network+policy_input['delta_add_network_fee']+policy_input['delta_cancel_network_fee']
    return 'treasury', max(n,0)

def s_agents_in_network(params, substep, state_history, previous_state,policy_input):
    agents=previous_state['number_of_agents']
    n_agents=agents+policy_input['agents_close'] + policy_input['agents_cancel']+policy_input['agents_create']
    return 'number_of_agents', max(n_agents,1000)

In [217]:
partial_state_update_blocks = [
    {   
        'policies': {
            'delta_create':  p_createBounty,
            'delta_add_network': p_createBounty, 
            'agents_create': p_createBounty,
            
            'delta_cancel': p_cancelBounty, 
            'delta_cancel_network':p_cancelBounty ,
            'delta_cancel_network_fee':p_cancelBounty,
            'agents_cancel': p_cancelBounty,
            
            'delta_close':p_closeBounty, 
            'delta_close_network':p_closeBounty,
            'delta_add_network_fee':p_closeBounty,
            'agents_close':p_closeBounty
            
        },
        'variables': {
            'bepro_in_network': s_bepro_in_network,
            'treasury': s_bepro_in_treasury,
            'bounties_in_progress':s_bounties_progress,
            'bounties_closed': s_bounties_closed,
            'number_of_agents':s_agents_in_network,
        }
    }
]

In [218]:
sim_config = config_sim({
    "N": 10, # the number of times we'll run the simulation ("Monte Carlo runs")
    "T": range(500), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

del configs[:] # Clear any prior configs

In [219]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [220]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (500, 6, 10, 5)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SubsetIDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Ns       : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
ExpIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Execution Mode: parallelized
Total execution time: 1.75s


In [221]:
# Convert raw results to a Pandas DataFrame
df = pd.DataFrame(raw_result)

# Insert cadCAD parameters for each configuration into DataFrame
for config in configs:
    # Get parameters from configuration
    parameters = config.sim_config['M']
    # Get subset index from configuration
    subset_index = config.subset_id
    
    # For each parameter key value pair
    for (key, value) in parameters.items():
        # Select all DataFrame indices where subset == subset_index
        dataframe_indices = df.eval(f'subset == {subset_index}')
        # Assign each parameter key value pair to the DataFrame for the corresponding subset
        df.loc[dataframe_indices, key] = value
df

,bounties_in_progress,bounties_closed,bepro_in_network,treasury,number_of_agents,simulation,subset,run,substep,timestep,network_fee,curator_fee,cancel_fee,prob_create,prob_cancel,prob_close
0,52.0,6.0,4884451.0,1.393131e+09,10000.0,0,0,1,0,0,0.05,0.03,0.01,0.1,0.01,0.03
1,52.0,6.0,4884451.0,1.393131e+09,10000.0,0,0,1,1,1,0.05,0.03,0.01,0.1,0.01,0.03
2,52.0,6.0,4884451.0,1.393131e+09,10000.0,0,0,1,1,2,0.05,0.03,0.01,0.1,0.01,0.03
3,52.0,6.0,4884451.0,1.393131e+09,10000.0,0,0,1,1,3,0.05,0.03,0.01,0.1,0.01,0.03
4,52.0,6.0,4884451.0,1.393131e+09,10000.0,0,0,1,1,4,0.05,0.03,0.01,0.1,0.01,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5005,123.0,103.0,39467451.0,1.394657e+09,10173.0,0,0,10,1,496,0.05,0.03,0.01,0.1,0.01,0.03
5006,123.0,103.0,39467451.0,1.394657e+09,10173.0,0,0,10,1,497,0.05,0.03,0.01,0.1,0.01,0.03
5007,121.0,105.0,38769451.0,1.394692e+09,10171.0,0,0,10,1,498,0.05,0.03,0.01,0.1,0.01,0.03
5008,125.0,105.0,38843451.0,1.394692e+09,10175.0,0,0,10,1,499,0.05,0.03,0.01,0.1,0.01,0.03


## Analysis

In [198]:
px.line(
    df,
    x='timestep', # Variable on the horizontal axis
    y=['bounties_in_progress', 'bounties_closed'], # Variables on the vertical axis
    #line_group='run', # One line for each MC run
    #facet_col='dt',
    log_y=False, 
    height=800 
)

In [199]:
px.line(
    df,
    x='timestep', # Variable on the horizontal axis
    y='number_of_agents', # Variable on the vertical axis
    color='run', # Color by MC run
    #facet_row='prey_birth_parameter', # Create a figure for each `prey_birth_parameter` parameter sweep
    #log_x=True, # Use log scale on the horizontal axis
    #log_y=True, # Use log scale on the vertical axis
    height=800,
)

In [222]:
px.line(
    df,
    x='timestep',
    y='treasury', 
    color='run', 
    height=800,
)

# Positive variation for treasury

In [223]:
px.line(
    df,
    x='timestep',
    y='bepro_in_network', 
    color='run', 
    height=800,
)

In [224]:
px.line(
    df,
    x='number_of_agents',
    y='bepro_in_network', 
    color='run', 
    height=800,
)